In [ ]:
#@title <-- Tap this if you on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldAI below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

# For users who don't have a GPU!

In [ ]:
# Set up the environment.
!git clone https://github.com/zyddnys/manga-image-translator
%cd manga-image-translator/
!python -m pip install -r requirements.txt
!python -m pip install git+https://github.com/lucasb-eyer/pydensecrf.git

# To expose service outside
!apt install nginx
!echo "daemon off; events { worker_connections 1024; } http { server { access_log /dev/stdout combined; autoindex_format json; listen 127.0.0.1:5004; server_name example.com; location /result { autoindex on; alias /content/manga-image-translator/result; } } }" > nginx.conf
!npm i localtunnel -g

In [ ]:
import psutil, subprocess, os

# Get a list of all processes that are listening on specified ports
ports = [5003, 5004]
for proc in psutil.process_iter():
    try:
        for conn in proc.connections():
            if conn.laddr.port in ports:
                print(f"Killing process {proc.pid}...")
                proc.kill()
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

proc = subprocess.Popen(['python', '-m', "manga_translator", "--verbose", "--mode", "web", "--use-cuda"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
ngproc = subprocess.Popen(['nginx', '-c', f"{os.getcwd()}/nginx.conf"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
ltproc = subprocess.Popen(['lt', '--port', "5003"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
ltngproc = subprocess.Popen(['lt', '--port', "5004"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

ltline = ltproc.stdout.readline().decode()[:-1].split()[-1]
print("Api:", ltline)
ltngline = ltngproc.stdout.readline().decode()[:-1].split()[-1]
print("Directory:", ltngline + "/result")

print("Runner IP:", subprocess.run(["curl", "ipv4.icanhazip.com"], stdout=subprocess.PIPE).stdout.decode())

while True:
  line = proc.stdout.readline().decode()
  if not line:
    break
  #the real code does filtering here
  print(line.rstrip())